## Prueba de modelos ##

### Version 4: Modelo con 2 DER, planificación anual, granularidad horaria ###

Si el modelo tiene que optimizar el balance por horas, tiene 8640 VD, pero sigue teniendo 2 VD binarias

In [1]:
%load_ext autoreload

In [2]:
%autoreload 2

In [3]:
import pandapower as pp

In [4]:
import pandas as pd

In [5]:
import pyomo.environ as pe

In [6]:
import math

In [7]:
import random

In [8]:
#import itertools

In [9]:
import matplotlib.pyplot as plt

In [10]:
import numpy as np

In [25]:
from ModelWriters.SimpleBusbar import SimpleModelWriter

In [26]:
import ModelWriters.Resources.ExtGrid  as ExtGrid

In [27]:
import ModelWriters.Resources.Load  as Load

In [28]:
import ModelWriters.Resources.PV  as PV

In [29]:
import ModelWriters.Resources.WT  as WT

In [30]:
import ModelWriters.Simulation.ElectricityCost as ElectricityCost

In [31]:
import ModelWriters.Simulation.Demand as Demand

In [32]:
import ModelWriters.Simulation.Solar as Solar

In [33]:
import ModelWriters.Simulation.Wind as Wind

In [34]:
#import ModelWriters.SimpleBusbar

In [35]:
net = pp.create_empty_network() 
b0 = pp.create_bus(net, vn_kv=13.2)

pp.create_ext_grid(net, bus=b0)

#ramal 1
b11 = pp.create_bus(net, vn_kv=13.2)
b12 = pp.create_bus(net, vn_kv=13.2)

pp.create_line(net, from_bus=b0, to_bus=b11, length_km=0.8, std_type="NAYY 4x50 SE")
pp.create_line(net, from_bus=b11, to_bus=b12, length_km=2.1, std_type="NAYY 4x50 SE")

pp.create_sgen(net, b11, p_mw=0.075, q_mvar=0.025, max_p_mw=0.15, max_q_mvar=0.15)
pp.create_load(net, bus=b12, p_mw=0.350)

#ramal 2
b21 = pp.create_bus(net, vn_kv=13.2)
pp.create_line(net, from_bus=b0, to_bus=b21, length_km=5.0, std_type="NAYY 4x50 SE")
pp.create_sgen(net, b21, p_mw=0.15, q_mvar=0.00, max_p_mw=0.40, max_q_mvar=0.10)
##pp.create_storage(net, bus=b21, p_mw=0.25, q_mvar=0.1, max_e_mwh = 3.0)

#ramal 3
b31 = pp.create_bus(net, vn_kv=13.2)
b32 = pp.create_bus(net, vn_kv=13.2)
pp.create_line(net, from_bus=b0, to_bus=b31, length_km=0.5, std_type="NAYY 4x50 SE")
pp.create_line(net, from_bus=b31, to_bus=b32, length_km=1.5, std_type="NAYY 4x50 SE")
pp.create_sgen(net, b31, p_mw=0.08, q_mvar=0.00, max_p_mw=0.30, max_q_mvar=0.10)
pp.create_load(net, bus=b32, p_mw=0.85, q_mvar=0.33)


#ramal 4
b41 = pp.create_bus(net, vn_kv=13.2)
pp.create_line(net, from_bus=b0, to_bus=b41, length_km=0.5, std_type="NAYY 4x50 SE")
pp.create_load(net, bus=b41, p_mw=0.22, q_mvar=0.05)
##pp.create_storage(net, bus=b41, p_mw=0.5, q_mvar=0.1, max_e_mwh = 5.0)



2

In [36]:
 pp.runpp(net)

In [37]:
print(net.res_bus.vm_pu)
print(net.res_line)

0    1.000000
1    0.999206
2    0.996501
3    1.002824
4    0.998497
5    0.993536
6    0.999583
Name: vm_pu, dtype: float64
   p_from_mw  q_from_mvar   p_to_mw     q_to_mvar     pl_mw   ql_mvar  \
0   0.276189    -0.058072 -0.275956  4.891292e-02  0.000233 -0.009159   
1   0.350956    -0.023913 -0.350000 -2.477944e-15  0.000956 -0.023913   
2  -0.149573    -0.057583  0.150000  1.065188e-15  0.000427 -0.057583   
3   0.775911     0.307919 -0.774624 -3.134920e-01  0.001287 -0.005573   
4   0.854624     0.313492 -0.850000 -3.300000e-01  0.004624 -0.016508   
5   0.220093     0.044267 -0.220000 -5.000000e-02  0.000093 -0.005733   

   i_from_ka   i_to_ka      i_ka  vm_from_pu  va_from_degree  vm_to_pu  \
0   0.012344  0.012268  0.012344    1.000000        0.000000  0.999206   
1   0.015398  0.015362  0.015398    0.999206       -0.015073  0.996501   
2   0.007010  0.006542  0.007010    1.000000        0.000000  1.002824   
3   0.036512  0.036605  0.036605    1.000000        0.000000  0.99

In [38]:
print(net)

This pandapower network includes the following parameter tables:
   - bus (7 elements)
   - load (3 elements)
   - sgen (3 elements)
   - ext_grid (1 element)
   - line (6 elements)
 and the following results tables:
   - res_bus (7 elements)
   - res_line (6 elements)
   - res_ext_grid (1 element)
   - res_load (3 elements)
   - res_sgen (3 elements)


In [39]:
net.sgen

,name,bus,p_mw,q_mvar,sn_mva,scaling,in_service,type,current_source,max_p_mw,max_q_mvar
0,None,1,0.075,0.025,NaN,1.0,True,wye,True,0.15,0.15
1,None,3,0.150,0.000,NaN,1.0,True,wye,True,0.40,0.10
2,None,4,0.080,0.000,NaN,1.0,True,wye,True,0.30,0.10


In [40]:
net.storage

,name,bus,p_mw,q_mvar,sn_mva,soc_percent,min_e_mwh,max_e_mwh,scaling,in_service,type


Agregamos los parámetros del modelo a los elementos de la red.

In [41]:
def net_add_optfw(net):
    #agregamos parámetros para el modelo de optimización
    tables = [net.ext_grid, net.load, net.sgen, net.storage]

    for t in tables: 
        #initial cost
        t['model'] = None


In [42]:
net_add_optfw(net)

Vemos como queda por ejemplo el elemento de red externa:

In [43]:
net.ext_grid

,name,bus,vm_pu,va_degree,in_service,model
0,None,0,1.0,0.0,True,None


Defino una función normal de Python para que nos va a dar la componente lineal del costo de la energía

Variables que define el modelo:

y para el año

d para el día del año

h para la hora del día

dt la granularidad del modelo, en horas


temp la temperatura

wv la velocidad del viento

I la irradiación solar


eg el crecimiento económico en pu respecto al año 0


Estas variables llegan como parámetros en forma de diccionario en el argumento 'model_status' (ver este nombre)

Todas las funciones reciben este argumento, la lógica de la función indica que valor retorna. Pr ejemplo, si la granularidad es 24 h, debe retornar el valor medio del parámetro producido.

In [44]:
m = ExtGrid.ExtGrid('SET')
m['pr_mw'] = 5.0
m['pa_pu'] = 1.0
m['oc_1_mu'] = ElectricityCost.fixed_scalonated_electricity_cost
net.ext_grid['model'][0] = m
net.ext_grid['name'][0] = 'SET'



In [45]:
net.ext_grid

,name,bus,vm_pu,va_degree,in_service,model
0,SET,0,1.0,0.0,True,Resource: SET


In [46]:
m = Load.Load('Tornquinst')
m['pa_pu'] = Demand.scalonated_seasoned_randomized_demand
m['pr_mw'] = net.load['p_mw'][0]
net.load['model'][0] = m

m = Load.Load('Bell Ville')
m['pa_pu'] = Demand.scalonated_seasoned_randomized_demand
m['pr_mw'] = net.load['p_mw'][1]
net.load['model'][1] = m

m = Load.Load('City Bell')
m['pa_pu'] = Demand.scalonated_seasoned_randomized_demand
m['pr_mw'] = net.load['p_mw'][2]
net.load['model'][2] = m

In [47]:
net.load

,name,bus,p_mw,q_mvar,const_z_percent,const_i_percent,sn_mva,scaling,in_service,type,model
0,None,2,0.35,0.00,0.0,0.0,NaN,1.0,True,wye,Resource: Tornquinst
1,None,5,0.85,0.33,0.0,0.0,NaN,1.0,True,wye,Resource: Bell Ville
2,None,6,0.22,0.05,0.0,0.0,NaN,1.0,True,wye,Resource: City Bell


In [48]:
m = PV.PVGenerator('PV1')
m.ic_0_mu = 2.0
m.ic_1_mu = 0.75
m.oc_0_mu = 0.0
m.oc_1_mu = 400e-6

net.sgen['name'][0] = m.name
net.sgen['model'][0] = m

m = PV.PVGenerator('PV2')
m.ic_0_mu = 3.0
m.ic_1_mu = 0.75
m.oc_0_mu = 0.0
m.oc_1_mu = 500e-6

net.sgen['name'][2] = m.name
net.sgen['model'][2] = m

In [49]:
m = WT.WTGenerator('WT1')
m.ic_0_mu = 5.0
m.ic_1_mu = 3.0
m.oc_0_mu = 0.0
m.oc_1_mu = 700e-6

net.sgen['name'][1] = m.name
net.sgen['model'][1] = m

In [50]:
net.sgen

,name,bus,p_mw,q_mvar,sn_mva,scaling,in_service,type,current_source,max_p_mw,max_q_mvar,model
0,PV1,1,0.075,0.025,NaN,1.0,True,wye,True,0.15,0.15,Resource: PV1
1,WT1,3,0.150,0.000,NaN,1.0,True,wye,True,0.40,0.10,Resource: WT1
2,PV2,4,0.080,0.000,NaN,1.0,True,wye,True,0.30,0.10,Resource: PV2


In [51]:
"""
net.storage['name'][0] = 'ST1'
net.storage['pa_pu'][0] = 1.0
net.storage['ic_0_mu'][0] = 0.5
net.storage['ic_1_mu'][0] = 1.0
net.storage['ic_1_mu_cap'][0] = 0.5
net.storage['oc_0_mu'][0] = 0.0
net.storage['oc_1_mu'][0] = 1200e-6
net.storage['sizeable'][0] = True

net.storage['name'][1] = 'ST2'
net.storage['pa_pu'][1] = 1.0
net.storage['ic_0_mu'][1] = 0.5
net.storage['ic_1_mu'][1] = 2.0
net.storage['ic_1_mu_cap'][1] = 7.0
net.storage['oc_0_mu'][1] = 0.0
net.storage['oc_1_mu'][1] = 1400e-6
net.storage['sizeable'][1] = True
"""

"\nnet.storage['name'][0] = 'ST1'\nnet.storage['pa_pu'][0] = 1.0\nnet.storage['ic_0_mu'][0] = 0.5\nnet.storage['ic_1_mu'][0] = 1.0\nnet.storage['ic_1_mu_cap'][0] = 0.5\nnet.storage['oc_0_mu'][0] = 0.0\nnet.storage['oc_1_mu'][0] = 1200e-6\nnet.storage['sizeable'][0] = True\n\nnet.storage['name'][1] = 'ST2'\nnet.storage['pa_pu'][1] = 1.0\nnet.storage['ic_0_mu'][1] = 0.5\nnet.storage['ic_1_mu'][1] = 2.0\nnet.storage['ic_1_mu_cap'][1] = 7.0\nnet.storage['oc_0_mu'][1] = 0.0\nnet.storage['oc_1_mu'][1] = 1400e-6\nnet.storage['sizeable'][1] = True\n"

In [52]:
net.storage

,name,bus,p_mw,q_mvar,sn_mva,soc_percent,min_e_mwh,max_e_mwh,scaling,in_service,type,model


## Construcción de Escenarios

In [66]:
dias = 365
años = 5

In [67]:
#el rango de horas:
T_i = range(24)

#el rango de dias:
Dias_i = range(dias)

Escenarios_i = range(len(T_i)*len(Dias_i))

In [68]:
m_s_base = [{'y': 0, 'd': 0, 'dd':1.0, 'h': 0.0, 'dt': 1.0, 'temp': 20.0, 'I':1000.0, 'wv': 10.0, 'eg': 1.0}]

registros = len(Dias_i)*len(T_i)

Escenarios =  pd.DataFrame(m_s_base, index = Escenarios_i)

reg = 0
for d in Dias_i:
    for h in T_i:
        Escenarios['d'][reg] = d
        Escenarios['h'][reg] = h
        Escenarios['I'][reg] = Solar.solar_irradiance_seasoned_randomized(d, h)
        Escenarios['wv'][reg] = random.uniform(1.0, 20.0)
        reg += 1

In [69]:
Escenarios

,y,d,dd,h,dt,temp,I,wv,eg
0,0,0,1.0,0.0,1.0,20.0,0.000000,15.150427,1.0
1,0,0,1.0,1.0,1.0,20.0,0.000000,16.498661,1.0
2,0,0,1.0,2.0,1.0,20.0,0.000000,10.689945,1.0
3,0,0,1.0,3.0,1.0,20.0,0.000000,13.777055,1.0
4,0,0,1.0,4.0,1.0,20.0,0.000000,13.773256,1.0
...,...,...,...,...,...,...,...,...,...
8755,0,364,1.0,19.0,1.0,20.0,210.898074,4.346594,1.0
8756,0,364,1.0,20.0,1.0,20.0,16.830537,1.048345,1.0
8757,0,364,1.0,21.0,1.0,20.0,0.000000,7.236712,1.0
8758,0,364,1.0,22.0,1.0,20.0,0.000000,11.221933,1.0


## Construcción del Modelo

Construccion automatica del modelo
Es un modelo de dimensionamiento de 3 DERs (WT+PV)

In [70]:
model_writer = SimpleModelWriter(net, Escenarios)

In [71]:
model = model_writer.create_model()

In [72]:
##model.pprint()

In [73]:
#solver_manager = pe.SolverManagerFactory('neos')
#result_obj = solver_manager.solve(model.model, opt='cplex')

opt = pe.SolverFactory('cplex')
#opt = pe.SolverFactory('glpk')
result_obj= opt.solve(model, tee=True)


Welcome to IBM(R) ILOG(R) CPLEX(R) Interactive Optimizer 20.1.0.0
  with Simplex, Mixed Integer & Barrier Optimizers
5725-A06 5725-A29 5724-Y48 5724-Y49 5724-Y54 5724-Y55 5655-Y21
Copyright IBM Corp. 1988, 2020.  All Rights Reserved.

Type 'help' for a list of available commands.
Type 'help' followed by a command name for more
information on commands.

CPLEX> Logfile 'cplex.log' closed.
Logfile 'C:\Users\jmsar\AppData\Local\Temp\tmpaudrk89w.cplex.log' open.
CPLEX> Problem 'C:\Users\jmsar\AppData\Local\Temp\tmpa51mcor0.pyomo.lp' read.
Read time = 0.09 sec. (3.38 ticks)
CPLEX> Problem name         : C:\Users\jmsar\AppData\Local\Temp\tmpa51mcor0.pyomo.lp
Objective sense      : Minimize
Variables            :   35047  [Nneg: 26284,  Box: 8760,  Binary: 3]
Objective nonzeros   :   35046
Linear constraints   :   35044  [Less: 26283,  Equal: 8761]
  Nonzeros           :   77926
  RHS nonzeros       :    8761

Variables            : Min LB: 0.000000         Max UB: 5.000000       
Objective n

In [ ]:
getattr(model_writer.model, 'PV1_create').value

In [74]:
model_writer.backconfigure_network()

In [75]:
model_writer.net.sgen

,name,bus,p_mw,q_mvar,sn_mva,scaling,in_service,type,current_source,max_p_mw,max_q_mvar,model
0,PV1,1,0.075,0.025,NaN,1.0,False,wye,True,0.000000,0.000000,Resource: PV1
1,WT1,3,0.150,0.000,NaN,1.0,True,wye,True,1.650665,0.825333,Resource: WT1
2,PV2,4,0.080,0.000,NaN,1.0,False,wye,True,0.000000,0.000000,Resource: PV2


In [76]:
model_writer.net.storage

,name,bus,p_mw,q_mvar,sn_mva,soc_percent,min_e_mwh,max_e_mwh,scaling,in_service,type,model


In [77]:
Resultados = model_writer.get_scenes_results()

In [78]:
Resultados

,SET_p_mw,Tornquinst_p_mw,Bell Ville_p_mw,City Bell_p_mw,WT1_p_mw
0,0.000000,-0.099754,-0.221189,-0.067021,0.387963
1,0.000000,-0.098661,-0.182962,-0.047791,0.329414
2,0.000000,-0.096002,-0.248742,-0.058697,0.403441
3,0.000000,-0.097155,-0.243654,-0.048407,0.389216
4,0.000000,-0.082972,-0.213894,-0.065682,0.362548
...,...,...,...,...,...
8755,1.556398,-0.515802,-0.977169,-0.248659,0.185231
8756,1.825480,-0.407196,-1.133770,-0.284515,0.000000
8757,1.042293,-0.516842,-0.870190,-0.238044,0.582783
8758,0.000000,-0.150860,-0.270099,-0.067548,0.488507


In [ ]:
# Set the plot curve with markers and a title
fig, ax = plt.subplots(1, 1)

fig.set_size_inches(10,6)
a=180*24
#b=360
b=180*24+5*24
ax.plot(Resultados['SET'][a:b], label = 'P Ext')
ax.plot(Resultados['PV2'][a:b], label = 'PV')
ax.plot(Resultados['ST1'][a:b], label = 'ST')
ax.plot(Resultados['E_ST1'][a:b], label = 'E_ST')
ax.plot(Resultados['Tornquinst'][a:b]+Resultados['Bell Ville'][a:b]+Resultados['City Bell'][a:b], label = 'Load')

#ax.plot(Resultados['time'][240:360], Resultados['p_mw_PV'][240:360], label = 'PV')
#ax.plot(Resultados['time'][240:360], Resultados['p_mw_Load'][240:360], label = 'Load')

ax.set_xlabel('time')
ax.set_ylabel('P[mw]')

ax.legend()

ax.grid(True)


plt.show()

In [ ]:
df = pd.DataFrame([], index = range(100))

In [ ]:
df['Col 1'] = [i for i in range(100)]

In [ ]:
df